Dieser Quellcode ist Bestandteil der Dissertation von Ines Reinecke
vorgelegt am 11.07.2023 der Technischen Universität Dresden, Medizinische Fakultät

Dieser Code enthält 

* Eingelesene Daten - Datensatz DS-Top1000
* Generieren von zwei Dataframes für die korrekten und nicht korrekten Ergebnisse von Algorithmus 3
* statistische Informationen generieren für die beiden neuen Dataframes in Bezug auf Algorithmus 3 und den Ähnlichkeitswert von Levenshtein
* Durchführung eines beidseitigen t-tests um zu prüfen, ob sich die Ähnlichkeitswerte des Algorithmus 3 für die beiden Ergebnismengen signifikant unterscheiden


In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import plotly.graph_objects as go
import plotly.express as px
from chord import Chord
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import random
from sklearn import preprocessing
from matplotlib_venn import venn2
from matplotlib_venn import venn3

In [2]:
#Einlesen der Daten, die häufigsten 1000 Freitexte der Medikationsverordnungen (Datensatz DS-Top1000)

df = pd.read_csv('../data_in/FINAL_TOP1000.csv',skipinitialspace=True, sep=";")


In [3]:
# Vorbereitung der Daten
# Änderung der folgenden Spalten zu nummerischen Werten

df["eval1"] = pd.to_numeric(df["eval1"])
df["eval2"] = pd.to_numeric(df["eval2"])
df["eval31"] = pd.to_numeric(df["eval31"])
df["eval32"] = pd.to_numeric(df["eval32"])
df["eval33"] = pd.to_numeric(df["eval33"])
df["LEV1"] = pd.to_numeric(df["LEV1"])
df["LEV2"] = pd.to_numeric(df["LEV2"])
df["LEV3"] = pd.to_numeric(df["LEV3"])

### Normalisierung des Dataframes
subset = df.filter(['LEV1','eval31'], axis=1)
subset['LEV1'] = subset.index
subset = subset.reset_index(drop=True)

scaler = preprocessing.MinMaxScaler()
names = subset.columns
d = scaler.fit_transform(subset)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df

#Gruppierung Datensatz DS-Top1000, basierend auf der Häufigkeit der ATC Codes die erkannt wurden von Algorithmus 3

groupedATC = df.groupby('STEP31').count()
groupedATC = groupedATC.sort_values('eval31', ascending=False)

In [4]:
#create two new datasets for wrong and corret results of algorithm 3, preparation for t-test 
#Generierung von zwei neuen Dataframes, die jeweils alle Einträge enthalten, die für Algorithmus 3 ein korrektes und ein inkorrektes Ergebnis erzeugt haben

#Dataframe mit den Einträgen, die FALSCH waren (Algorithmus 3)
lev31FALSE = df.loc[df['eval31'] == 0.0]

#Dataframe mit den Einträgen, die RICHTIG waren (Algorithmus 3)
lev31TRUE = df.loc[df['eval31'] == 1.0]

In [5]:
#Generieren der statistischen Kenngrößen für das Dataframe mit den korrekten Ergebnissen von Algorithmus 3
#(Anzahl, Durchschnitt, Standardabweichung, Min, Max und die Quantile)
#bezogen auf den Ähnlichkeitwert

lev31TRUE["LEV1"].describe(include='all')

count    766.000000
mean      84.284595
std       14.856875
min       21.000000
25%       76.000000
50%       87.000000
75%       96.000000
max      100.000000
Name: LEV1, dtype: float64

In [6]:
#Generieren der statistischen Kenngrößen für das Dataframe mit den korrekten Ergebnissen von Algorithmus 3
#(Anzahl, Durchschnitt, Standardabweichung, Min, Max und die Quantile)
#bezogen auf den Ähnlichkeitwert

lev31FALSE["LEV1"].describe(include='all')

count    234.000000
mean      67.183761
std       15.520594
min       29.000000
25%       55.000000
50%       63.000000
75%       75.000000
max      100.000000
Name: LEV1, dtype: float64

In [7]:
# Änderung der Einträge in der Spalte "eval31" von 1.0 auf true und 0.0 auf false
# Vorbereitung der Durchführung des t-tests 

true = df[df['eval31']==1.0]
false = df[df['eval31']==0.0]

# Zweiseitiger t-Test, um zu prüfen, ob sich die beiden Gruppen, wahr und falsch, auf der Grundlage des Levenshtein Ähnlichkeitswerte signifikant unterscheiden

ergebnis_ttest = ttest_ind(true['LEV1'], false['LEV1'], equal_var=False)

print("Ergebnis t-test Vergleich der beiden Ergebnismengen wenn Algorithmus 3 korrekt oder falsch gearbeitet hat: ",ergebnis_ttest)


Ergebnis t-test Vergleich der beiden Ergebnismengen wenn Algorithmus 3 korrekt oder falsch gearbeitet hat:  Ttest_indResult(statistic=14.897946966519024, pvalue=1.0360007176052793e-39)
